In [1]:
from transformers import pipeline
import pandas as pd
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report
import torch
import numpy as np


In [2]:
# --- 1. Load zero-shot classifier ---
classifier = pipeline(
    "zero-shot-classification",
    model="MoritzLaurer/deberta-v3-large-zeroshot-v2.0",
    device=0 if torch.cuda.is_available() else -1,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/870M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/970 [00:00<?, ?B/s]

Device set to use cpu


In [3]:
# --- 2. Candidate labels ---
candidate_labels = [
    "shopping for software or tools",           # High Intent
    "hiring employees or growing team",         # Medium Intent
    "discussing HR trends or articles",         # Low Intent
    "off-topic, greeting, hate, spam, personal" # No Intent
]

In [4]:
# --- 3. Sample labeled test dataset ---
data = [
    ("Looking for an HR automation tool", "High Intent"),
    ("Hiring a new marketing manager", "Medium Intent"),
    ("Top HR trends for 2025", "Low Intent"),
    ("I love cooking pasta on weekends", "No Intent"),
    ("Need a recruitment AI solution", "High Intent"),
    ("Our company is expanding the dev team", "Medium Intent")
]

In [5]:

df = pd.DataFrame(data, columns=["text", "label"])

In [6]:
# --- 4. Run zero-shot predictions ---
preds = []
scores = []

In [7]:
for text in df['text']:
    result = classifier(text, candidate_labels, multi_label=False)
    top_label = result['labels'][0]
    score = result['scores'][0]

    # Map top_label back to our category names
    if "shopping" in top_label:
        mapped_label = "High Intent"
    elif "hiring" in top_label:
        mapped_label = "Medium Intent"
    elif "discussing" in top_label:
        mapped_label = "Low Intent"
    else:
        mapped_label = "No Intent"

    preds.append(mapped_label)
    scores.append(score)

df['predicted'] = preds
df['score'] = scores


In [8]:
# --- 5. Evaluation metrics ---
accuracy = accuracy_score(df['label'], df['predicted'])
print("Accuracy:", accuracy)

Accuracy: 0.6666666666666666


In [9]:
# ROC-AUC (multi-class, one-vs-rest)
# Convert to one-hot for ROC-AUC
labels_map = {"High Intent":0, "Medium Intent":1, "Low Intent":2, "No Intent":3}
y_true = np.array([labels_map[l] for l in df['label']])
y_pred_prob = np.zeros((len(df), len(labels_map)))
for i, p in enumerate(df['predicted']):
    y_pred_prob[i, labels_map[p]] = df['score'][i]

roc_auc = roc_auc_score(pd.get_dummies(y_true), y_pred_prob, multi_class='ovr')
print("ROC-AUC:", roc_auc)

# Classification report
print("\nClassification Report:")
print(classification_report(df['label'], df['predicted']))

ROC-AUC: 0.8125

Classification Report:
               precision    recall  f1-score   support

  High Intent       1.00      0.50      0.67         2
   Low Intent       1.00      1.00      1.00         1
Medium Intent       0.50      1.00      0.67         2
    No Intent       0.00      0.00      0.00         1

     accuracy                           0.67         6
    macro avg       0.62      0.62      0.58         6
 weighted avg       0.67      0.67      0.61         6



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
